In [4]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()
secrets = [s.strip() for s in os.getenv("phi_3").split(",")]

url = secrets[0]
key = secrets[1]

api_url = url + "/v1/chat/completions"

headers = {
    "Authorization": f"Bearer {key}",
    "Content-Type": "application/json",
}

In [6]:
def stream_response(headers, payload):
    with requests.post(api_url, json=payload, headers=headers, stream=True) as response:
        response.raise_for_status()
        
        for line in response.iter_lines():
            if line:
                decoded_line = line.decode("utf-8").strip()
                
                if decoded_line.startswith("data: "):
                    json_data = decoded_line[6:].strip()
                    if json_data and json_data != "[DONE]":
                        try:
                            parsed_data = json.loads(json_data)
                            if "choices" in parsed_data and parsed_data["choices"]:
                                chunk = parsed_data["choices"][0].get("delta", {}).get("content", "")
                                if chunk:
                                    print(chunk, end="", flush=True)
                        except json.JSONDecodeError:
                            continue

        print()

def non_stream_response(headers, payload):
    
    response = requests.post(api_url, json=payload, headers=headers)
    response.raise_for_status() # Ensure the request was successful

    response_json = response.json()
    
    if "choices" in response_json and response_json["choices"]:
        generated_text = response_json["choices"][0].get("message", {}).get("content", "")
        print(generated_text)
        return generated_text
    else:
        print("Unexpected response format:", response_json)
        return None

def inference(prompt, stream=True):
    
    payload = {
        "frequency_penalty": 0,
        "max_tokens": 2048,
        "messages": [{"role": "user", "content": prompt}],
        "presence_penalty": 0,
        "stream": stream,
        "temperature": 0.8,
        "top_p": 0.5
    }
    if stream:
        return stream_response(headers, payload)
    else:
        return non_stream_response(headers, payload)

inference("What's the meaning of life?")


The meaning of life is a philosophical question concerning the significance of existence or consciousness. Humans have pondered this question for centuries, and it has been addressed by various disciplines, including philosophy, religion, science, and literature.

Different perspectives offer various answers to the meaning of life:

1. Religious and spiritual perspectives: Many religions provide their own answers to the meaning of life. For example, in Christianity, the meaning of life is to love and serve God and to attain eternal life with Him. In Buddhism, the meaning of life is to achieve enlightenment and escape the cycle of rebirth.

2. Philosophical perspectives: Philosophers have proposed numerous theories about the meaning of life. Existentialists, such as Jean-Paul Sartre, argue that life has no inherent meaning, and it is up to each individual to create their own purpose. In contrast, Aristotle believed that the meaning of life is to achieve eudaimonia, or human flourishing,

In [ ]:
#